In [1]:
# PYTORCH VERSION OF MNIST MODEL
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [2]:
class MHISTDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None):
        self.img_dir = img_dir
        self.labels = pd.read_csv(labels_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels.iloc[idx, 0])  # Image names from the first column
        image = Image.open(img_name)

        # Map 'hp' and 'ssa' to integer labels
        label = 0 if self.labels.iloc[idx, 1] == 'HP' else 1  # Assuming 'HP' -> 0 and 'SSA' -> 1

        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
# Define the transformations
augmentation = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor()
])

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Path to images and the CSV file
img_dir = './data/images'
labels_file = './data/annotations.csv'

# Load the CSV file and split based on the 'Partition' column
annotations = pd.read_csv(labels_file)
train_data = annotations[annotations['Partition'] == 'train']
test_data = annotations[annotations['Partition'] == 'test']

# Save the train and test splits into separate CSV files (if needed)
train_data.to_csv('train_annotations.csv', index=False)
test_data.to_csv('test_annotations.csv', index=False)

# Create Dataset instances
train_dataset = MHISTDataset(img_dir=img_dir, labels_file='train_annotations.csv', transform=augmentation)
test_dataset = MHISTDataset(img_dir=img_dir, labels_file='test_annotations.csv', transform=transform)

# Create DataLoader instances
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [4]:
# Define the model based off of MHIST
class MHISTModel(nn.Module):
    def __init__(self):
        super(MHISTModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=7, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(10, 10, kernel_size=7, padding=0)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
        self.fc1 = nn.Linear(10 * 11 * 11, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Conv -> Relu -> Pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Conv -> Relu -> Pooling
        x = x.view(x.size(0), -1)  # Flatten into 1-D
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.sigmoid(x)  # Output with sigmoid activation
        return x

In [5]:
# Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 100

# Model, loss function, and optimizer
model = MHISTModel()
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler
scheduler = ExponentialLR(optimizer, gamma=0.91)

# Training and evaluation loop
for epoch in range(epochs):
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training loop
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        labels = labels.float().unsqueeze(1)  # Reshape labels to (batch_size, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the training loss
        running_train_loss += loss.item()

        # Calculate training accuracy
        predicted = (outputs > 0.5).float()  # Classify predictions
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    # Average training loss and accuracy
    avg_train_loss = running_train_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Testing loop
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            labels = labels.float().unsqueeze(1)  # Reshape labels to (batch_size, 1)
            loss = criterion(outputs, labels)

            # Accumulate the test loss
            running_test_loss += loss.item()

            # Calculate testing accuracy
            predicted = (outputs > 0.5).float()  # Classify predictions
            correct_test += (predicted == labels).sum().item()
            total_test += labels.size(0)

    # Average testing loss and accuracy
    avg_test_loss = running_test_loss / len(test_loader)
    test_accuracy = 100 * correct_test / total_test

    # Adjust learning rate based on scheduler
    scheduler.step()

    # Print epoch, training loss, training accuracy, testing loss, and testing accuracy
    print(f'Epoch [{epoch + 1}/{epochs}] - '
          f'Training Loss: {avg_train_loss:.4f} - Training Accuracy: {train_accuracy:.2f}% - '
          f'Testing Loss: {avg_test_loss:.4f} - Testing Accuracy: {test_accuracy:.2f}%')

Epoch [1/100] - Training Loss: 0.6169 - Training Accuracy: 70.80% - Testing Loss: 0.6646 - Testing Accuracy: 63.15%
Epoch [2/100] - Training Loss: 0.6073 - Training Accuracy: 71.03% - Testing Loss: 0.6720 - Testing Accuracy: 63.15%
Epoch [3/100] - Training Loss: 0.6122 - Training Accuracy: 71.03% - Testing Loss: 0.6719 - Testing Accuracy: 63.15%
Epoch [4/100] - Training Loss: 0.6103 - Training Accuracy: 71.03% - Testing Loss: 0.6624 - Testing Accuracy: 63.15%
Epoch [5/100] - Training Loss: 0.6075 - Training Accuracy: 71.03% - Testing Loss: 0.6622 - Testing Accuracy: 63.15%
Epoch [6/100] - Training Loss: 0.6068 - Training Accuracy: 71.03% - Testing Loss: 0.6635 - Testing Accuracy: 63.15%
Epoch [7/100] - Training Loss: 0.6044 - Training Accuracy: 71.03% - Testing Loss: 0.6690 - Testing Accuracy: 63.15%
Epoch [8/100] - Training Loss: 0.6071 - Training Accuracy: 71.03% - Testing Loss: 0.6687 - Testing Accuracy: 63.15%
Epoch [9/100] - Training Loss: 0.6057 - Training Accuracy: 71.03% - Test

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import time
import copy

# Use pre-trained ResNet-50 and modify the final layer for the number of classes
num_classes = 2
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model.fc = nn.Linear(model.fc.in_features, num_classes)
# Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.91)

# Function to train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=100):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()  # Set model to evaluate mode
                dataloader = test_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloader:

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has the best accuracy
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        # Step the learning rate scheduler after each epoch
        scheduler.step()

        print()

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=100)

Epoch 1/100
----------
Train Loss: 0.6066 Acc: 0.6777
Test Loss: 0.5991 Acc: 0.6520

Epoch 2/100
----------
Train Loss: 0.5298 Acc: 0.7301
Test Loss: 0.5591 Acc: 0.7001

Epoch 3/100
----------
Train Loss: 0.4783 Acc: 0.7490
Test Loss: 0.5190 Acc: 0.7083

Epoch 4/100
----------
Train Loss: 0.4570 Acc: 0.7747
Test Loss: 0.5113 Acc: 0.7329

Epoch 5/100
----------
Train Loss: 0.4404 Acc: 0.7954
Test Loss: 0.4925 Acc: 0.7503

Epoch 6/100
----------


KeyboardInterrupt: 